In [22]:
import os
import google.genai as genai
from dotenv import load_dotenv

In [23]:
# .env 로드 및 api키 전역에 설정.
load_dotenv() 
google_key = os.getenv("GEMINI_API_KEY")

MODEL = "model/gemini-2.5-flash"

In [24]:
FILE_NAME = "data2.xlsx"

In [26]:
# 파일 import 및 저장 경로 설정
from pathlib import Path

PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_PATH = PROJECT_ROOT / "downloads" / FILE_NAME
MD_PATH = PROJECT_ROOT / "debug" / "parsed_table.md"

In [27]:
# excel or csv의 raw data를 markdown으로 파싱

import importlib
import utils.parsing as parsing

importlib.reload(parsing)

from utils.parsing import parse_workbook_all_sheets_to_markdown


md_all = parse_workbook_all_sheets_to_markdown(
    DATA_PATH,
    max_total_chars=300_000,
    max_rows_per_sheet=2000,
    save_combined_markdown=False,
    combined_markdown_path=MD_PATH,
)


In [28]:
# 파싱된 마크다운 확인
md_all

'## Sheet: 표지\n\n| Unnamed: 0                                                                   | Unnamed: 6   | Unnamed: 8    |\n|:-----------------------------------------------------------------------------|:-------------|:--------------|\n| 2024년 제주특별자치도                                                        | nan          | nan           |\n|  외국인관광객 입도통계                                                       |              |               |\n| nan                                                                          | 제주관광공사 | 연구조사팀    |\n| nan                                                                          | nan          | 064) 740-6049 |\n| * 제주특별자치도관광협회 입도통계 및 한국관광공사 한국관광통계에서 발췌∙수정 | nan          | nan           |\n| * 제주특별자치도 입도통계는 확정치가 발표되지 않은 경우 잠정치를 이용함      | nan          | nan           |\n\n\n## Sheet: 1월\n\n| 제주 외국인관광통계 1월   | Unnamed: 1   | Unnamed: 2   | Unnamed: 3   | Unnamed: 4   | Unnamed: 5          | Unnamed: 6   | Unnamed: 7   | Unnamed: 8          |

In [30]:
# load prompts
from utils.prompt_loader import load_json_generator_prompts, load_report_generator_prompts

json_generator_prompts = load_json_generator_prompts()
report_generator_prompts = load_report_generator_prompts()

/Users/anseonghui/text-to-json/prompt


In [8]:
json_generator_prompts

'You are a prompt engineer for extracting JSON from Excel tables.\n\nINPUTS YOU WILL RECEIVE:\n1) An Excel table parsed as Markdown text (may include line breaks, multirow headers, merged-like cells, NaNs, and wrapped text).\n\nYOUR TASK:\nProduce TWO OUTPUTS:\n\nA) "LLM_EXTRACTION_PROMPT"\nWrite a single, self-contained prompt intended for a separate LLM (the “Extractor LLM”). The prompt must instruct the Extractor LLM to:\n- Parse the provided LaTeX table as the authoritative representation of the Excel.\n- Infer a reasonable JSON structure from the table contents (no schema is provided).\n- Use headers as keys where possible and preserve units in values.\n- Handle messy cases: multiline cells, missing values, repeated headers, footnotes, totals rows, and ambiguous columns.\n- Output ONLY valid JSON (no commentary, no markdown, no metadata).\n- If the JSON structure includes any narrative fields such as "report", "summary", "analysis", or descriptive text fields, those fields MUST be

In [9]:
report_generator_prompts

'[SYSTEM]\n    SYSTEM:\nYou generate three artifacts from the same content:\n1) a human-readable professional report,\n2) a JSON representation of that report,\n3) a JSON Schema that describes exactly that JSON.\n\nCRITICAL CONSTRAINTS:\n- The JSON must be derivable ONLY from the report text.\n- Do NOT assume spreadsheets, tables, cells, or any external source.\n- Do NOT introduce any concepts, fields, or entities that do not appear in the report.\n- The JSON Schema must describe the structure of the produced JSON and NOTHING MORE.\n- The schema MAY differ for every output.\n- If a concept does not appear in the report, it must NOT appear in the JSON or schema.\n- Every JSON field must be supported by one or more sentences in the report.\n\nOUTPUT FORMAT:\nReturn exactly three sections in this order:\n=== REPORT ===\n=== JSON ===\n=== JSON_SCHEMA ===\nDo not use code fences.\n\n\n    [DOCUMENT STYLE]\n    [PosixPath(\'/Users/anseonghui/text-to-json/prompt/prompt_report.txt\')]\n\n    [

In [31]:
report_prompt = report_generator_prompts + '''=======INPUT MARKDOWN======{md_all}'''

In [32]:
import os
from google import genai

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])  
resp = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=report_prompt
)

In [33]:
# 생성한 보고서를 확인을 위해 저장
p = Path("debug/report_debug.txt")

p.parent.mkdir(parents=True, exist_ok=True)

normalized =resp.text.replace("\r\n", "\n").replace("\r", "\n")

p.write_text(normalized, encoding="utf-8")

2766

In [34]:
from utils.parsing_answer import parse_report_json_schema

data = parse_report_json_schema(resp.text)

In [41]:
p = Path(f'../data/report/{FILE_NAME}.txt')

p.parent.mkdir(parents=True, exist_ok=True)
normalized = data["report"].replace("\r\n", "\n").replace("\r", "\n")
p.write_text(normalized, encoding="utf-8")

import json

p = Path(f'../data/json/{FILE_NAME}.json')

p.parent.mkdir(parents=True, exist_ok=True)
normalized = data["json_obj"]
p.write_text(json.dumps(normalized, ensure_ascii=False, indent=2), encoding="utf-8")


p = Path(f'../data/json_schema/{FILE_NAME}.json')

p.parent.mkdir(parents=True, exist_ok=True)
normalized = data["json_schema"]
p.write_text(json.dumps(normalized, ensure_ascii=False, indent=2), encoding="utf-8")

1879

In [ ]:
data["report"]


'## 보고서\n\n본 보고서는 제공된 데이터를 기반으로 작성되었으며, 명확성과 전문성을 최우선으로 합니다. 보고서의 모든 내용은 제공된 정보에 근거하며, 추가적인 해석이나 가정을 포함하지 않습니다.\n\n### 프로젝트 정보\n\n프로젝트 이름은 "프로젝트 블루"입니다. 이 프로젝트는 현재 진행 중이며, 2023년 3월 15일에 시작되었습니다. 프로젝트의 상태는 "진행 중"으로 보고되었습니다.\n\n### 팀 구성\n\n프로젝트 팀은 다음과 같이 구성됩니다. 팀 리더는 김민지입니다. 팀 멤버로는 이서연과 박지훈이 있습니다. 각 팀원의 역할에 대한 구체적인 정보는 제공되지 않았습니다.\n\n### 예산 정보\n\n프로젝트에 할당된 총 예산은 50,000달러입니다. 현재까지 지출된 금액은 25,000달러입니다. 남은 예산에 대한 정보는 제공되지 않았습니다.\n\n### 위험 요소\n\n현재 프로젝트에는 두 가지 위험 요소가 식별되었습니다. 첫 번째 위험 요소는 "자원 부족"이며, 두 번째 위험 요소는 "일정 지연"입니다. 이러한 위험 요소에 대한 구체적인 해결 방안은 제공되지 않았습니다.'

In [19]:
data["json_obj"]

{'프로젝트 정보': {'프로젝트 이름': '프로젝트 블루', '시작일': '2023년 3월 15일', '상태': '진행 중'},
 '팀 구성': {'팀 리더': '김민지', '팀 멤버': ['이서연', '박지훈']},
 '예산 정보': {'총 예산': '50,000달러', '지출액': '25,000달러'},
 '위험 요소': ['자원 부족', '일정 지연']}

In [20]:
data["json_schema"]

{'$schema': 'https://json-schema.org/draft/2020-12/schema',
 'type': 'object',
 'properties': {'프로젝트 정보': {'type': 'object',
   'properties': {'프로젝트 이름': {'type': 'string'},
    '시작일': {'type': 'string'},
    '상태': {'type': 'string'}},
   'required': ['프로젝트 이름', '시작일', '상태']},
  '팀 구성': {'type': 'object',
   'properties': {'팀 리더': {'type': 'string'},
    '팀 멤버': {'type': 'array', 'items': {'type': 'string'}}},
   'required': ['팀 리더', '팀 멤버']},
  '예산 정보': {'type': 'object',
   'properties': {'총 예산': {'type': 'string'}, '지출액': {'type': 'string'}},
   'required': ['총 예산', '지출액']},
  '위험 요소': {'type': 'array', 'items': {'type': 'string'}}},
 'required': ['프로젝트 정보', '팀 구성', '예산 정보', '위험 요소']}